In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
from tqdm import tqdm

In [ ]:
headers = {
    'Accept': 'text/html, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'Cookie': '_cc_id=7d432df57a4f31cd17bc5844585041a5; dable_uid=58396953.1714037666383; _ga_400SVTKXBC=GS1.1.1718849154.4.1.1718849295.60.0.0; __gads=ID=5b281e9c4283ec0f:T=1715328023:RT=1718958800:S=ALNI_MZcPQFiSr1LNMtAdgsKr58hDTAyAA; __gpi=UID=00000e15df75b3ed:T=1715328023:RT=1718958800:S=ALNI_MYdK77xtqHw6xYeM6NIEYefLpJq-Q; cto_bundle=fRcqU19qbGVudkc4bTNEYUQ2ZXliRml2dEd3JTJGSmFLSyUyQnBvcFJzMXV1WFQ3OHk1YkNLTzJrMkZpMkVRaTZCVGZQODFEc0FpVHQlMkJadjZsT0RxUFJlbWhOZTBXUHE2RVhoZDRsZ0hEJTJCd3VZbnFDSkwzT3QxY0p5UHFwJTJCcCUyRkhuYlZtRWpqSWxrSWxPOHFjMG1pZHlFcnp4VTZreXclM0QlM0Q; cto_bidid=RILlZ19EVDg2SEZpS1VQYVl3YnRTOHd0eHlIUzNCOUE3UVVDWGMlMkZUbjkyQ2d0UmlBYzlFd3olMkZnQzR0Q2p4c3U4aDElMkZMMHlrSmhtT2x2SVFPQTJlQVd0dFhKTmVRRXhtbzg0VUY1aUdGWWpVR2tDTSUzRA; cto_dna_bundle=Um-akV9qbGVudkc4bTNEYUQ2ZXliRml2dEczSyUyRlhaZDdxdjcxaFRVdmlqTSUyQlNZd1dNeHV3TDdZMVY5RXdJcFRzT25xS3BPOUhCcU55bjhhb00xWHpodUxHMWclM0QlM0Q; isShowLogin=true; ASP.NET_SessionId=unzgpaxvrv1lyc020xz0iemo; __RequestVerificationToken=zTt098g2W4Cgh8bGrrqGHeeGbH5iBIAX5SbtCfdKzbKDklmfcs362E63p9k7hFEotHxLNgU_lF10wtFbllkANaStyemHwCHgQPGytqlEM0Y1; _gid=GA1.2.495137277.1731040596; _ga=GA1.1.362242690.1715049946; CookieNewsFirt=0; _ga_EXMM0DKVEX=GS1.1.1731056569.41.1.1731056571.58.0.0',
    'Origin': 'https://vietstock.vn',
    'Referer': 'https://vietstock.vn/chung-khoan.htm',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}


In [111]:
categ_dict = {
    'chung khoan':144,
    'doanh nghiep':733,
    'bds':763,
    'tai chinh':734,
    'hang hoa':2,
    'kinh te':5307,
    'the gioi': 736,
    'tai chinh ca nhan': 4259,
    'dong duong': 1317
}
max_pages = {
    144:16,
    733:19,
    763:7,
    734:7,
    2:6,
    5307:11,
    736: 10,
    4259: 3,
    1317: 1
}

In [106]:
def checkDate(publishDate, channelId):
    for time in publishDate:
        try:
            time_convert = datetime.strptime(time).timestamp()
            print(time_convert)
            print(datetime.strptime('30/11 00:00', '%d/%m %H:%M').timestamp())
            if time_convert < datetime.strptime('30/11 00:00', '%d/%m %H:%M').timestamp():
                print(f'Min date reached for: ', list(categ_dict.keys())[list(categ_dict.values()).index(channelId)])
                return
        except:
            next

In [113]:
all_urls = []
for channelId in list(categ_dict.values()):
    for page in range(1, max_pages[channelId] + 1):
        data = {
            'channelID': channelId,
            'page': page,
        }
        response = requests.post('https://vietstock.vn/StartPage/ChannelContentPage', headers=headers, data=data)
        html = BeautifulSoup(response.text, 'html.parser')
        # publishDate = [pdate.find_all('a')[1].get_text().strip() for pdate in html.find_all('div', {'class':'meta3'})]
        # try:
        #     checkDate(publishDate, channelId)
        # except:
        #     publishDate = html.find('div', {'class':'meta3'}).find_all('a')[1].get_text().strip()
        urls = ['https://vietstock.vn' + a.find('a').get('href') for a in html.find_all('div', {'class':'img_wrap'})]
        all_urls.extend(urls)

In [92]:
def get_article(url, headers):
    html = BeautifulSoup(
        requests.get(url, headers=headers).content,
        'html.parser'
    )
    title = html.find('h1', {'class':'article-title'}).get_text().strip()
    date = html.find('span',{'class':'date'}).get_text().strip()
    desc = html.find('p', {'class':'pHead'}).get_text().strip().replace("\xa0", " ")
    categ = html.find('div', {'class':'bridcrumb'}).find_all('span', {'itemprop':'name'})[-1].get_text().strip()
    body = html.find('div', {'itemprop':'articleBody'}).get_text().strip().replace("\xa0", " ").replace("\n", " ")
    return {
        'title': title, 'date': date, 'desc':desc, "categ":categ, 'body':body, 'source':'Vietstock'
    }

In [114]:
dates = []
sources = []
urls = []
titles = []
descs = []
categs = []
bodies = []

for link in tqdm(all_urls):
    try:
        data = get_article(link, headers)
        dates.append(data["date"])
        sources.append(data["source"])
        urls.append(link)
        titles.append(data["title"])
        descs.append(data["desc"])
        categs.append(data["desc"])
        bodies.append(data['body'])
    except Exception as error:
        print(f"Error at:, {link} {error}")

vietstock_data = pd.DataFrame({
        'date': dates,
        'source' : sources,
        'url' : urls,
        'title' : titles,
        'description' : descs,
        'category' : categs,
        'body' : bodies
    })

 58%|█████▊    | 461/800 [03:55<02:46,  2.04it/s]

Error at:, https://vietstock.vn/2024/11/mb-30-nam-30-trieu-khach-hang-va-nhung-gia-tri-lon-lao-757-1242730.htm 'NoneType' object has no attribute 'get_text'


 88%|████████▊ | 705/800 [05:56<00:44,  2.13it/s]

Error at:, https://vietstock.vn/2024/11/cac-ty-phu-my-chon-phe-nao-trong-cuoc-dua-vao-nha-trang-2024-775-1243093.htm 'NoneType' object has no attribute 'get_text'


 97%|█████████▋| 776/800 [06:30<00:09,  2.50it/s]

Error at:, https://vietstock.vn/2024/11/nhung-con-so-an-tuong-shark-tank-mua-7-4262-1240762.htm 'NoneType' object has no attribute 'get_text'


100%|██████████| 800/800 [06:41<00:00,  1.99it/s]


In [115]:
vietstock_data.to_csv("vietstock_data.csv")

In [116]:
vietstock_data

,date,source,url,title,description,category,body
0,29 phút trước,Vietstock,https://vietstock.vn/2024/11/tuan-11-15112024-...,Tuần 11-15/11/2024: 10 cổ phiếu nóng dưới góc...,Các cổ phiếu nóng được phân tích trong báo cáo...,Các cổ phiếu nóng được phân tích trong báo cáo...,Tuần 11-15/11/2024: 10 cổ phiếu nóng dưới góc ...
1,44 phút trước,Vietstock,https://vietstock.vn/2024/11/nhip-dap-thi-truo...,Nhịp đập Thị trường 11/11: Tiếp tục giằng co,"Đầu phiên 11/11, tính tới 9h30, VN-Index giằng...","Đầu phiên 11/11, tính tới 9h30, VN-Index giằng...",Nhịp đập Thị trường 11/11: Tiếp tục giằng co Đ...
2,58 phút trước,Vietstock,https://vietstock.vn/2024/11/pho-chu-tich-pgn-...,Phó Chủ tịch PGN hoàn tất bán hơn 26% vốn,"Ông Trần Đặng Công - Phó Chủ tịch HĐQT, Tổng G...","Ông Trần Đặng Công - Phó Chủ tịch HĐQT, Tổng G...",Phó Chủ tịch PGN hoàn tất bán hơn 26% vốn Ông...
3,1 giờ trước,Vietstock,https://vietstock.vn/2024/11/quy-ngoai-tu-oman...,Quỹ ngoại từ Oman chuyển đổi trái phiếu sang g...,Theo báo cáo của CTCP Đầu tư Văn Phú - Invest ...,Theo báo cáo của CTCP Đầu tư Văn Phú - Invest ...,Quỹ ngoại từ Oman chuyển đổi trái phiếu sang g...
4,1 giờ trước,Vietstock,https://vietstock.vn/2024/11/khi-su-am-dam-va-...,Khi sự ảm đạm và nghi ngờ đang cùng xuất hiện ...,Đo lường về vòng quay thanh khoản của HOSE cho...,Đo lường về vòng quay thanh khoản của HOSE cho...,Khi sự ảm đạm và nghi ngờ đang cùng xuất hiện ...
...,...,...,...,...,...,...,...
792,15/09/2024 20:02,Vietstock,https://vietstock.vn/2024/09/thi-truong-gao-ca...,Thị trường gạo Campuchia sắp đón nhận thêm nhi...,"Bộ trưởng Bộ Thương mại Campuchia (MoC), bà Ch...","Bộ trưởng Bộ Thương mại Campuchia (MoC), bà Ch...",Thị trường gạo Campuchia sắp đón nhận thêm nhi...
793,14/09/2024 14:02,Vietstock,https://vietstock.vn/2024/09/chung-khoan-campu...,"Chứng khoán Campuchia đón thêm 55,000 tài khoả...",Theo công bố của Sở Giao dịch Chứng khoán Camp...,Theo công bố của Sở Giao dịch Chứng khoán Camp...,"Chứng khoán Campuchia đón thêm 55,000 tài khoả..."
794,09/09/2024 20:02,Vietstock,https://vietstock.vn/2024/09/kinh-te-campuchia...,Kinh tế Campuchia đang ổn định nhờ xuất khẩu,Nền kinh tế Campuchia đang thể hiện những dấu ...,Nền kinh tế Campuchia đang thể hiện những dấu ...,Kinh tế Campuchia đang ổn định nhờ xuất khẩu ...
795,31/08/2024 06:34,Vietstock,https://vietstock.vn/2024/08/vang-the-gioi-gia...,Vàng thế giới giảm sau báo cáo lạm phát của Mỹ,"Giá vàng giảm vào ngày thứ Sáu (30/08), do đồn...","Giá vàng giảm vào ngày thứ Sáu (30/08), do đồn...",Vàng thế giới giảm sau báo cáo lạm phát của Mỹ...
